In [1]:
showPlots = False
defaultFigsize = (12, 6)

# Introduzione

Si vuole calcolare il numero di riproduzione effettivo $R$ nel tempo $t$ dato il numero noto di nuovi casi $k$ in $t$.

Si userà la seguente notazione:

- $X \in (a, b)$: $a < X < b$

- $X \in [a, b]$: $a \leq X \leq b$

- $X \in (a, b]$: $a < X \leq b$

- $X \in [a, b)$: $a \leq X < b$

- $X_i$: valore della variabile $X$ al tempo $i$

  - $R_t$: valore di $R$ in $t$
  
  - $k_t$: valore di $k$ in $t$

- $P(X)$: probabilità **a priori** che si verifichi $X$

- $P(X|Y^*)$: probabilità **a posteriori** che si verifichi $X$ noto il valore di $Y^*$

- $\mathscr{L}(Y^*|X)$: verosimiglianza (likelihood) del verificarsi noto di $Y^*$ data la variabile aleatoria $X$

- $\mathbf{X}_{n \times m}$: matrice (o vettore) $\mathbf{X}$ di $n$ righe e $m$ colonne

- $| \mathbf{X} |$: modulo del vettore $\mathbf{X}$

- $\mathscr{N}(\mu,\sigma)$: distributione normale (Gaussiana) avente media $\mu$ e deviazione standard $\sigma$

- $\mathscr{P}(X,Y)$: distribuzione di Poisson di $X$ con parametro $Y$

## Metodo

Il metodo utilizzato è una versione modificata da Kevin Systrom (2020) <cite data-cite="k-sys"></cite> di una soluzione presentata da Bettencourt e Ribeiro (2008) <cite data-cite="bettencourt_ribeiro_2008"></cite>.

Si calcola la probabilità **a posteriori** del verificarsi di $R_t$ dato $k_t^*$ in $t$:

$$ P(R_t|k_t^*) $$

dove con $k^*$ si indica che è l'evento noto (già accaduto).

Per il Teorema di Bayes:

\begin{equation}\label{eq:bayestheorem}
P(A|B^*) = \frac{P(B^*|A) \cdot P(A)}{P(B^*)}
\end{equation}

quindi

\begin{equation}\label{eq:formula1}
P(R_t|k_t^*) = \frac{P(k_t^*|R) \cdot P(R_t)}{P(k_t^*)}
\end{equation}

sapendo che

\begin{equation}\label{eq:denominatore}
P(k_t^*) = \sum_{i=0}^{t} \bigg( P(k_i|R) \cdot P(R_i) \bigg)
\end{equation}

***

Quindi, per ogni $t$ si calcolerà:

- $P(k_t^*|R)$: la probabilità **a posteriori** di aver osservato $k_t^*$ nuovi casi dati i possibili valori di $R$, primo termine del numeratore nella \eqref{eq:formula1}

- $P(R_t)$: la probabilità **a priori** dei possibili valori di $R_t$ in $t$ in base al valore calcolato del giorno precedente $R_{t-1}$, secondo termine del numeratore nella \eqref{eq:formula1}

- $P(k_t^*)$: la probabilità **a priori** di osservare $k_t^*$ nuovi casi in generale (pari alla sommatoria dei numeratori precedenti), denominatore \eqref{eq:denominatore} della \eqref{eq:formula1}

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as sps
from scipy.interpolate import interp1d
import scipy.integrate as integrate

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.colors import ListedColormap
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.patches import Polygon

from IPython.display import display, HTML, Markdown, Latex
%matplotlib inline

In [3]:
def latexplot(plt, figlabel, figcaption, showme=False):

    figname = figlabel + '.png'
    plt.savefig(figname, bbox_inches='tight')

    if showPlots or showme:
        plt.show()
        return
    plt.close()

    strLatex=fr"""
    \begin{{figure}}
    \centering
        \includegraphics{{{figname}}}
        \caption{{{figcaption}}}
        \label{{fig:{figlabel}}}
    \end{{figure}}"""
    
    return display(Latex(strLatex)) 

# Costanti

## gamma

È noto che $R_t$ dipende da $\gamma$, l'inverso dell'intervallo seriale $T_{serial}$ ovvero il tempo medio tra casi successivi in una catena di trasmissione, spesso calcolato come intervallo tra presunta infezione e comparsa dei sintomi (periodo di contagiosità asintomatico). Per Covid-19 è stato stimato intorno ai $7.5$ giorni <cite data-cite="sanche2020high"></cite>:

$$ T_{\textrm{serial}} = 7.5 \; \textrm{days} $$

\begin{equation}\label{eq:gamma}
\gamma = \frac{1}{T_{\textrm{serial}}}
\end{equation}

In [4]:
T_serial = 7.5
gamma = 1 / T_serial
display(Markdown(f"$$\gamma \simeq {gamma:.4f}$$"))

$$\gamma \simeq 0.1333$$

## vettori R

Si crea un vettore $\mathbf{R}_{1\times1201}$ di tutti i possibili valori di $R$ da $R_{min}$, pari a $0$, ad $R_{max}$ per il quale si può assumere il valore massimo $12$ (figura $\ref{fig:vettorer}$).

$$ R_{min} = 0 ,\; R_{max} = 12 $$

$$ \mathbf{R} = \begin{Bmatrix}
R_{min} & R_{min}+0.01 & \cdots & R_{max}
\end{Bmatrix}_{1\times1201} $$

In [5]:
Rt_min = 0
Rt_max = 12
Rt_bra = np.linspace(Rt_min, Rt_max, Rt_max*100+1)
display(Markdown(fr"$$\mathbf{{R}} = \begin{{Bmatrix}} {Rt_bra[0]:.2f} & {Rt_bra[1]:.2f} & \cdots & {Rt_bra[-2]:.2f} & {Rt_bra[-1]:.2f} \end{{Bmatrix}}_{{1\times1201}}$$"))
display(Markdown(fr"$$| \mathbf{{R}} | = {len(Rt_bra)}$$"))

$$\mathbf{R} = \begin{Bmatrix} 0.00 & 0.01 & \cdots & 11.99 & 12.00 \end{Bmatrix}_{1\times1201}$$

$$| \mathbf{R} | = 1201$$

$$ \mathbf{R}^{T} = \begin{Bmatrix}
R_{min}\\ 
\vdots\\ 
R_{max}
\end{Bmatrix}_{1201 \times 1} $$

In [6]:
ax = pd.Series(Rt_bra).plot(label="$\mathbf{R}$", figsize=defaultFigsize, title="Vettore $\mathbf{R}$")
ax.legend(loc="best", fontsize=18)
ax.set_ylabel("$R$")
latexplot(plt, "vettorer", "Vettore $\mathbf{R}$.")

<IPython.core.display.Latex object>

In [7]:
Rt_ket = Rt_bra[:, None]

# Valori noti

Si crea un vettore $\mathbf{k_t}_{1\times \omega}$ di tutti i $k$ nuovi casi per giorno da $t=0$ fino a oggi $t=\omega$ (figura $\ref{fig:kt}$). I dati italiani sono estratti dalla pubblicazione del Dipartimento di Protezione Civile <cite data-cite="pcm_dpc_2020"></cite>.

$$ t \in [0, \omega] $$

$$ \textrm{Nuovi casi al giorno } k_t $$

$$ \mathbf{k_t} = \begin{Bmatrix}
k_0 & k_1 & \cdots & k_\omega
\end{Bmatrix}_{1\times\omega} $$

In [8]:
def smooth(y, win=7, std=2):
    dy = y.diff()
    dy_smoothed = dy.rolling(win,
            win_type='gaussian',
            min_periods=1,
            center=True).mean(std=std).round()
    return dy_smoothed, dy

ita = None
url_ita = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv'
ita = pd.read_csv(url_ita,
                     usecols=['data', 'totale_casi'],
                     parse_dates=['data'],
                     index_col=['data'],
                     squeeze=True).sort_index()

k_bra, orig = smooth(ita, win=14, std=2.5)
display(Markdown(fr"$$\mathbf{{k_t}} = \begin{{Bmatrix}} {k_bra.values[0]:.0f} & \cdots & {k_bra.values[-1]:.0f} \end{{Bmatrix}}_{{1 \times {len(k_bra)}}}$$"))
display(Markdown(fr"$$| \mathbf{{k_t}} | = {k_bra.size}$$"))

ax = k_bra.plot(figsize=defaultFigsize, label="$\mathbf{k_t}$", title="Curva smussata dei $k_t$ nuovi casi/giorno")
ax.plot(orig, c="k", ls=":")
for day in orig.index:
    if day.strftime("%w") == "1":
        ax.axvline(day, ls="-", c="y", alpha=.25)
ax.legend(loc="best", fontsize=18)

latexplot(plt, "kt", r"Curva smussata con metodo media mobile gaussiana ($\sigma=2.5$) dei nuovi casi per giorno in Italia. I valori $k_t$ formano il vettore $\mathbf{k_t}_{1 \times \omega}$",
          showme=False)

$$\mathbf{k_t} = \begin{Bmatrix} 143 & \cdots & 1066 \end{Bmatrix}_{1 \times 79}$$

$$| \mathbf{k_t} | = 79$$

<IPython.core.display.Latex object>

# P(k|R)

Per risolvere $P(k_t^*|R)$ si nota anzitutto che, data una $\lambda$ serie di possibili $k$ nuovi casi

\begin{equation}\label{eq:poisson}
P(k|\lambda) = \mathscr{P}(k, \lambda) = \frac{\lambda^k \cdot e^{-\lambda}}{k!}
\end{equation}

ovvero, la probabilità **a posteriori** di avere $k$ nuovi casi dati $\lambda$ possibili valori $k$ è una distribuzione di Poisson di $k$ con parametro $\lambda$.

Sapendo però che il valore di $k$ è noto (tutti i valori di $k^*$ sono nel vettore $\mathbf{k_t}$) si può dire che

\begin{equation}\label{eq:likelihoods}
P(k^*|\lambda) \propto \mathscr{L}(\lambda|k^*)
\end{equation}

ovvero, la probabilità di *aver avuto* $k^*$ nuovi casi date $\lambda$ possibilità è proporzionale alla verosimiglianza delle $\lambda$ possibilità noto $k^*$.

Dalla letteratura, è nota la relazione tra $\lambda$, $k_t$ ed $R_t$ <cite data-cite="bettencourt_ribeiro_2008"></cite> derivante dalle equazioni differenziali del modello **SIR**:

\begin{equation}\label{eq:lambda}
\lambda = k_t \cdot \exp{\big[ \gamma (R_t - 1) \big]}
\end{equation}

Grazie a questa relazione è possibile dire che 

\begin{equation}\label{eq:pkr}
P(k_t^*|R) = P(k_t^*|\lambda)
\end{equation}

e costruire una matrice $\Lambda$ di $\omega-1$ colonne (perché il primo giorno è di outcome) e $1201$ righe contenente per ogni giorno $t$ tutti i possibili valori di $\lambda$ dati i possibili $1201$ valori di $R$ (tabella $\ref{tab:lambda}$ e figura $\ref{fig:lambda}$).

\begin{equation}\label{eq:matricelambda}
\Lambda = \mathbf{k_t} \cdot \exp\left[ \gamma ( \mathbf{R}^{T} - 1) \right], \; t=\left[ 0,\omega \right)
\end{equation}

$$ \Lambda = \begin{Bmatrix}
k_0 \cdot e^{\gamma (R_{min} - 1) } & \cdots  & k_{\omega - 1} \cdot e^{\gamma (R_{min} - 1) } \\ 
\vdots & \vdots & \vdots \\ 
k_0 \cdot e^{\gamma (R_{max} - 1) } & \cdots  &  k_{\omega - 1} \cdot e^{\gamma (R_{max} - 1) }  
\end{Bmatrix}_{1201 \times \omega-1} $$

In [9]:
lam = k_bra[:-1].values * np.exp(gamma * (Rt_ket - 1))

In [10]:
display(
    Latex(
        fr"""
\begin{{table}}
  \begin{{center}}
    \caption{{Estratto di matrice $\Lambda$: ${len(lam)} \times {len(lam[0])}$}}
    \label{{tab:lambda}}
    \begin{{tabular}}{{c|c|c|c|c|c|c|c|}}
          & {k_bra.index[0].strftime('%Y-%m-%d')} & 
          {k_bra.index[1].strftime('%Y-%m-%d')} &
          {k_bra.index[2].strftime('%Y-%m-%d')} &
          $\cdots$ & 
          {k_bra.index[-3].strftime('%Y-%m-%d')} & 
          {k_bra.index[-2].strftime('%Y-%m-%d')} & 
          {k_bra.index[-1].strftime('%Y-%m-%d')} \\
        \toprule
        $R=0.00$ & {lam[0,0]:.2f} & {lam[0,1]:.2f} & {lam[0,2]:.2f} & $\cdots$ & {lam[0,-3]:.2f} & {lam[0,-2]:.2f} & {lam[0,-1]:.2f} \\
        \midrule
        $R=0.01$ & {lam[1,0]:.2f} & {lam[1,1]:.2f} & {lam[1,2]:.2f} & $\cdots$ & {lam[1,-3]:.2f} & {lam[1,-2]:.2f} & {lam[1,-1]:.2f} \\
        \midrule
        $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ \\
        \midrule
        $R=11.99$ & {lam[1199,0]:.2f} & {lam[1199,1]:.2f} & {lam[1199,2]:.2f} & $\cdots$ & {lam[1199,-3]:.2f} & {lam[1199,-2]:.2f} & {lam[1199,-1]:.2f} \\
        \midrule
        $R=12.00$ & {lam[1200,0]:.2f} & {lam[1200,1]:.2f} & {lam[1200,2]:.2f} & $\cdots$ & {lam[1200,-3]:.2f} & {lam[1200,-2]:.2f} & {lam[1200,-1]:.2f} \\
        \bottomrule
    \end{{tabular}}
  \end{{center}}
\end{{table}}
        """
    )
)

<IPython.core.display.Latex object>

In [11]:
fig, ax = plt.subplots(figsize=defaultFigsize)
k_bra[:-1].plot(c="k", lw=1, label="$\mathbf{k_t}$")
ax.fill_between(
    k_bra.index[:-1], lam[0], lam[-1],
    label=r"$\mathbf{\Lambda}$",
    alpha=.5
)
ax.set_title("Matrice del parametro $\lambda$")
ax.legend(loc="best", fontsize=15)
latexplot(plt, "lambda", r"Valori del parametro $\lambda$, ovvero matrice $\Lambda_{1201 \times \omega-1}$ sulla serie temporale. "+
          "Per ogni giorno i valori $\lambda$ rappresentano tutti i possibili valori di $k_t$ dato il range di $R$ definito dal vettore $\mathbf{R} \in [R_{min},R_{max}]$."
         )

<IPython.core.display.Latex object>

Noti i valori della matrice $\Lambda$ si può dunque applicare la distribuzione di Poisson al vettore  $\mathbf{k_t}$ parametrizzata su $R$ con $t \in (0,\omega]$ e ottenere così la matrice $\mathbf{P\big(k_t^* \;\big|\; R\big)}$ di tutte le verosimiglianze dei possibili $1201$ valori di $R$ dati gli $\omega-1$ valori di $k_t^*$ (figura $\ref{fig:likelihoods}$ e tabella $\ref{tab:likelihoods}$):

\begin{equation}\label{eq:matricepkr}
P\big( \mathbf{k_t^*} \; \big| \; \Lambda \big)
= \mathscr{P} \big( \mathbf{k_t^*}, \Lambda \big) = 
\frac{\Lambda^{\mathbf{k_t^*}} \cdot e^{-\Lambda}}{\mathbf{k_t^*}!} =
\mathbf{P\big( k_t^* \;\big|\; R \big)}_{1201 \times \omega-1}
\end{equation}

In [12]:
likelihoods = pd.DataFrame(
    data = sps.poisson.pmf(k_bra[1:].values, lam),
    index = Rt_bra,
    columns = k_bra.index[1:])

likelihoods_norm = likelihoods / np.sum(likelihoods, axis=0)

In [13]:
display(
    Latex(
        fr"""
\begin{{table}}
  \begin{{center}}
    \caption{{Estratto di matrice $\mathbf{{P\big( k_t^* \;\big|\; R \big)}}$ (valori normalizzati): ${len(likelihoods_norm)} \times {len(likelihoods_norm.iloc[0])}$}}
    \label{{tab:likelihoods}}
    \begin{{tabular}}{{c|c|c|c|c|c|c|c|}}
          & {likelihoods_norm.columns[0].strftime('%Y-%m-%d')} & 
          {likelihoods_norm.columns[1].strftime('%Y-%m-%d')} &
          {likelihoods_norm.columns[2].strftime('%Y-%m-%d')} &
          $\cdots$ & 
          {likelihoods_norm.columns[-3].strftime('%Y-%m-%d')} & 
          {likelihoods_norm.columns[-2].strftime('%Y-%m-%d')} & 
          {likelihoods_norm.columns[-1].strftime('%Y-%m-%d')} \\
        \toprule
        $R=0.00$ & {likelihoods_norm.iloc[0,0]:.2e} & {likelihoods_norm.iloc[0,1]:.2e} & {likelihoods_norm.iloc[0,2]:.2e} & $\cdots$ & {likelihoods_norm.iloc[0,-3]:.2e} & {likelihoods_norm.iloc[0,-2]:.2e} & {likelihoods_norm.iloc[0,-1]:.2e} \\
        \midrule
        $R=0.01$ & {likelihoods_norm.iloc[1,0]:.2e} & {likelihoods_norm.iloc[1,1]:.2e} & {likelihoods_norm.iloc[1,2]:.2e} & $\cdots$ & {likelihoods_norm.iloc[1,-3]:.2e} & {likelihoods_norm.iloc[1,-2]:.2e} & {likelihoods_norm.iloc[1,-1]:.2e} \\
        \midrule
        $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ \\
        \midrule
        $R=11.99$ & {likelihoods_norm.iloc[1199,0]:.2e} & {likelihoods_norm.iloc[1199,1]:.2e} & {likelihoods_norm.iloc[1199,2]:.2e} & $\cdots$ & {likelihoods_norm.iloc[1199,-3]:.2e} & {likelihoods_norm.iloc[1199,-2]:.2e} & {likelihoods_norm.iloc[1199,-1]:.2e} \\
        \midrule
        $R=12.00$ & {likelihoods_norm.iloc[1200,0]:.2e} & {likelihoods_norm.iloc[1200,1]:.2e} & {likelihoods_norm.iloc[1200,2]:.2e} & $\cdots$ & {likelihoods_norm.iloc[1200,-3]:.2e} & {likelihoods_norm.iloc[1200,-2]:.2e} & {likelihoods_norm.iloc[1200,-1]:.2e} \\
        \bottomrule
    \end{{tabular}}
  \end{{center}}
\end{{table}}
        """
    )
)

<IPython.core.display.Latex object>

In [14]:
fig, ax = plt.subplots(figsize=defaultFigsize)
for x in range(5):
    ax.plot(
        likelihoods_norm.loc[:,likelihoods_norm.columns[x]].index,
        likelihoods_norm.loc[:,likelihoods_norm.columns[x]].values,
        label=likelihoods_norm.columns[x].strftime("%Y-%m-%d")
    )
ax.set_xlim(0,6)
ax.legend(loc="best")
ax.set_title(r"$\mathscr{P} \mathbf{(k_t^*, \Lambda)} \propto \mathbf{P( k_t^* \;|\; R )}$")
ax.set_xlabel("$R$")
latexplot(plt, "likelihoods", r"Verosimiglianze $\mathbf{P( k_t^* \;|\; R )}$ dei primi 5 giorni di dati registrati in Italia. " +
          r"Valori normalizzati. Nella $\eqref{eq:formula1}$ non servirà normalizzare le verosimiglianze perché il denonimatore stesso ha funzione di normalizzazione. " +
          "Si nota come ogni giorno diminuisca la deviazione standard e dunque si guadagni progressivamente in confidenza."
         )

<IPython.core.display.Latex object>

In [15]:
fig, ax = plt.subplots(figsize=(6, 3.5))
ax.set_xlim(0, 6)
ax.set_ylim(0, .035)
line, = ax.plot([], [])
plt.close(fig)

def init():
    ax.plot([], [])

def animate(i):
    line.set_data(lh[i].index, lh[i].values)
    line.set_label(labels[i])
    ax.legend(loc="upper right", fontsize=12)
    return (line,)

lh = []
labels = []
for day in likelihoods.columns:
    lh.append(likelihoods.loc[:,day])
    labels.append(f"$t=${day.strftime('%Y-%m-%d')}")

ax.set_title("$\mathscr{{L}}(\lambda|k_t^*) \propto P(k_t^*|R)$")
ax.set_xlabel("$R$")

anim = FuncAnimation(fig, animate, init_func=init, interval=5e2, frames=likelihoods.columns.size);

#HTML(anim.to_jshtml())

# P(R)

La probabilità **a priori** dei possibili valori $R$ può essere interpretata come una ditribuzione normale (Gaussiana) avente media $\mu = R$ e deviazione standard $\sigma$ che può essere stimata a $0.25$.

Si può dunque creare una matrice $\mathbf{P(R)}$ di $1201 \times 1201$ i cui elementi $P(n)_m$ di riga $n$ e colonna $m$ sono la probabilità di $R=n$ con $\mu=m$ (tabella $\ref{tab:pr}$, figure $\ref{fig:pr1}$ e $\ref{fig:pr2}$):

\begin{equation}\label{eq:pr}
\mathbf{P(R)} = \mathbf{\mathscr{N}(R,\sigma)} = \\
= 
\begin{Bmatrix}
P(0)_{0} & \cdots & P(0)_{6} & \cdots  & P(0)_{12} \\ 
\vdots & \ddots & \vdots & & \vdots \\
P(6)_{0} & \cdots & P(6)_{6} & \cdots & P(6)_{12} \\ 
\vdots & & \vdots & \ddots & \vdots \\
P(12)_{0} & \cdots & P(12)_{6} & \cdots & P(12)_{12}
\end{Bmatrix}_{1201 \times 1201}
\end{equation}

In [16]:
sigma = .25
process_matrix = sps.norm(
    loc=Rt_bra,
    scale=sigma
).pdf(Rt_ket)
process_matrix /= process_matrix.sum(axis=0)
#pd.DataFrame(process_matrix)

In [17]:
display(
    Latex(
        fr"""
\begin{{table}}
  \begin{{center}}
    \caption{{Estratto di matrice $\mathbf{{P(R)}}$: ${len(process_matrix)} \times {len(process_matrix[0])}$}}
    \label{{tab:pr}}
    \resizebox{{\textwidth}}{{!}} {{
        \begin{{tabular}}{{c|c|c|c|c|c|c|c|c|c|}}
              & $\mu=0.00$ &$\mu=0.01$ & $\mu=0.02$ & $\cdots$ & $\mu=6.00$ & $\cdots$ & $\mu=11.98$ & $\mu=11.99$ & $\mu=12.00$ \\
            \toprule
            $R=0.00$ & {process_matrix[0,0]:.2e} & {process_matrix[0,1]:.2e} & {process_matrix[0,2]:.2e} & $\cdots$ & {process_matrix[0,600]:.2e} & $\cdots$ & {process_matrix[0,-3]:.2e} & {process_matrix[0,-2]:.2e} & {process_matrix[0,-1]:.2e} \\
            \midrule
            $R=0.01$ & {process_matrix[1,0]:.2e} & {process_matrix[1,1]:.2e} & {process_matrix[1,2]:.2e} & $\cdots$ & {process_matrix[1,600]:.2e} & $\cdots$ & {process_matrix[1,-3]:.2e} & {process_matrix[1,-2]:.2e} & {process_matrix[1,-1]:.2e} \\
            \midrule
            $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ \\
            \midrule
            $R=6.00$ & {process_matrix[600,0]:.2e} & {process_matrix[600,1]:.2e} & {process_matrix[600,2]:.2e} & $\cdots$ & {process_matrix[600,600]:.2e} & $\cdots$ & {process_matrix[600,-3]:.2e} & {process_matrix[600,-2]:.2e} & {process_matrix[600,-1]:.2e} \\
            \midrule
            $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ \\
            \midrule
            $R=11.99$ & {process_matrix[1199,0]:.2e} & {process_matrix[1199,1]:.2e} & {process_matrix[1199,2]:.2e} & $\cdots$ & {process_matrix[1199,600]:.2e} & $\cdots$ & {process_matrix[1199,-3]:.2e} & {process_matrix[1199,-2]:.2e} & {process_matrix[1199,-1]:.2e} \\
            \midrule
            $R=12.00$ & {process_matrix[1200,0]:.2e} & {process_matrix[1200,1]:.2e} & {process_matrix[1200,2]:.2e} & $\cdots$ & {process_matrix[1200,600]:.2e} & $\cdots$ & {process_matrix[1200,-3]:.2e} & {process_matrix[1200,-2]:.2e} & {process_matrix[1200,-1]:.2e} \\
            \bottomrule
        \end{{tabular}}
    }}
  \end{{center}}
\end{{table}}
        """
    )
)

<IPython.core.display.Latex object>

In [46]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, ax = plt.subplots(figsize=(7,7))
ax.set_title("Matrice $\mathbf{P(R)}$")
ax.set_xticks([0, 200, 400, 600, 800, 1000, 1200])
ax.set_yticks([0, 200, 400, 600, 800, 1000, 1200])
ax.set_xticklabels([0, 0, 2, 4, 6, 8, 10, 12])
ax.set_yticklabels([0, 0, 2, 4, 6, 8, 10, 12])
im = ax.matshow(process_matrix, cmap="Blues")
#ax.axhline(600, ls=":")
#ax.axvline(600, ls=":")
ax.set_xlabel("$\mu$", fontsize="xx-large")
ax.set_ylabel("R", fontsize="xx-large")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
latexplot(plt, "pr1", r"Matrice $\mathbf{P(R)}$. Il colore indica la densità di probabilità. " +
          r"Essendo gaussiana, è concentrata intorno a $\mu$ ovvero sulla diagonale della matrice.",
          showme=False
         )

<IPython.core.display.Latex object>

In [19]:
process_matrix_df = pd.DataFrame(process_matrix)
fig, ax = plt.subplots(figsize=defaultFigsize)
for x in [15, 225, 575, 805, 1012]:
    ax.plot(
        process_matrix_df[x].index,
        process_matrix_df[x].values,
        label=f"$\mu={x/100:.2f}$",
        ls="-.", lw=2
    )
    ax.axvline(x, ls=":", c="k")
ax.set_xticks([0, 200, 400, 600, 800, 1000, 1200])
ax.set_xticklabels([0, 2, 4, 6, 8, 10, 12])
ax.legend(loc="best")
ax.set_title("$\mathscr{N}(R,\sigma)$")
ax.set_xlabel("$R$")
ax.set_ylim(0, 0.025)
ax.set_xlim(0, 1200)
latexplot(plt, "pr2", r"Distribuzione gaussiana di $P(R)$ con $\sigma=0.25$. Esempio per alcuni valori di $\mu$.",
          showme=False)

<IPython.core.display.Latex object>

In [20]:
fig, ax = plt.subplots(figsize=(6, 3.5))
ax.set_xlim(0, 1200)
ax.set_xticks([0, 200, 400, 600, 800, 1000, 1200])
ax.set_xticklabels([0, 2, 4, 6, 8, 10, 12])
ax.set_ylim(0, .035)
ax.set_xlabel("R")
line, = ax.plot([], [])
plt.close(fig)

process_matrix_df = pd.DataFrame(process_matrix)

def init():
    ax.plot([], [])

def animate(i):
    line.set_data(process_matrix_df[i*10].index, process_matrix_df[i*10].values)
    line.set_label(f"$\mu = {i/10:.1f}$")
    ax.legend(loc="upper left", fontsize=15)
    return (line,)


ax.set_title("$P(R,\sigma)$")

anim = FuncAnimation(fig, animate, init_func=init, interval=10, frames=process_matrix_df.columns.size//10+1);

#HTML(anim.to_jshtml())

# P(R|k)

È dunque possibile ora creare una matrice vuota $\mathbf{P\big(R_t \;\big|\; k_t^*\big)}$ di $1201 \times \omega$ i cui elementi $P(R_n)_m$ di riga $n$ e colonna $m$ saranno i valori della probabilità di $R=m$ al tempo $t=n$ (tabella $\ref{tab:prkempty}$).

\begin{equation}\label{eq:matriceprk}
\mathbf{P\big(R_t \;\big|\; k_t^*\big)} = 
\begin{Bmatrix}
P(R_0)_{0} &  P(R_1)_{0} & \cdots  & P(R_\omega)_{0} \\ 
\vdots & \vdots & & \vdots \\
P(R_0)_{6} &  P(R_1)_{6} & \cdots & P(R_\omega)_{6} \\ 
\vdots & \vdots &  & \vdots \\
P(R_0)_{12} & P(R_1)_{12} & \cdots & P(R_\omega)_{12}
\end{Bmatrix}_{1201 \times \omega}
\end{equation}

Non avendo alcuna informazione precedente a $t=0$, si imposta la probabilità iniziale della prima colonna $\mathbf{P\big(R_0 \;\big|\; k_0^*\big)}$ ovvero degli elementi $P(R_0)_m$ come costante in modo che $\sum P(R_0)_m = 1$:

In [21]:
display(
    Markdown(fr"""
$$P(R_0)_{{m}} = \frac{{1}}{{ | \mathbf{{R}} | }} = \frac{{1}}{{{len(Rt_bra)}}} = {1/len(Rt_bra):.2e}, \; \forall m \in [R_{{min}}, R_{{max}}]$$ 
    """
    )
)


$$P(R_0)_{m} = \frac{1}{ | \mathbf{R} | } = \frac{1}{1201} = 8.33e-04, \; \forall m \in [R_{min}, R_{max}]$$ 
    

In [22]:
prior0 = np.ones_like(Rt_bra)/len(Rt_bra)
prior0 /= prior0.sum()
#prior0

In [23]:
# Create a DataFrame that will hold our posteriors for each day
# Insert our prior as the first posterior.
posteriors = pd.DataFrame(
    index=Rt_bra,
    columns=k_bra.index,
    data={k_bra.index[0]: prior0}
)

In [24]:
display(
    Latex(
        fr"""
\begin{{table}}
  \begin{{center}}
    \caption{{Estratto di matrice vuota $\mathbf{{P(R_t|k_t^*)}}$: ${len(posteriors)} \times {len(posteriors.iloc[0])}$}}
    \label{{tab:prkempty}}
    \begin{{tabular}}{{c|c|c|c|c|c|c|c|}}
          & {posteriors.columns[0].strftime('%Y-%m-%d')} & 
          {posteriors.columns[1].strftime('%Y-%m-%d')} &
          {posteriors.columns[2].strftime('%Y-%m-%d')} &
          $\cdots$ & 
          {posteriors.columns[-3].strftime('%Y-%m-%d')} & 
          {posteriors.columns[-2].strftime('%Y-%m-%d')} & 
          {posteriors.columns[-1].strftime('%Y-%m-%d')} \\
        \toprule
        $R=0.00$ & {posteriors.iloc[0,0]:.2e} & {posteriors.iloc[0,1]:.2e} & {posteriors.iloc[0,2]:.2e} & $\cdots$ & {posteriors.iloc[0,-3]:.2e} & {posteriors.iloc[0,-2]:.2e} & {posteriors.iloc[0,-1]:.2e} \\
        \midrule
        $R=0.01$ & {posteriors.iloc[1,0]:.2e} & {posteriors.iloc[1,1]:.2e} & {posteriors.iloc[1,2]:.2e} & $\cdots$ & {posteriors.iloc[1,-3]:.2e} & {posteriors.iloc[1,-2]:.2e} & {posteriors.iloc[1,-1]:.2e} \\
        \midrule
        $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ \\
        \midrule
        $R=11.99$ & {posteriors.iloc[1199,0]:.2e} & {posteriors.iloc[1199,1]:.2e} & {posteriors.iloc[1199,2]:.2e} & $\cdots$ & {posteriors.iloc[1199,-3]:.2e} & {posteriors.iloc[1199,-2]:.2e} & {posteriors.iloc[1199,-1]:.2e} \\
        \midrule
        $R=12.00$ & {posteriors.iloc[1200,0]:.2e} & {posteriors.iloc[1200,1]:.2e} & {posteriors.iloc[1200,2]:.2e} & $\cdots$ & {posteriors.iloc[1200,-3]:.2e} & {posteriors.iloc[1200,-2]:.2e} & {posteriors.iloc[1200,-1]:.2e} \\
        \bottomrule
    \end{{tabular}}
  \end{{center}}
\end{{table}}
        """
    )
)

<IPython.core.display.Latex object>

Per ogni giorno $t \in (0, \omega]$ si calcola la probabilità di $R_t$ basata sulla distribuzione normale del giorno precedente $t-1$

\begin{equation}\label{eq:prgiorno}
\begin{gathered}
\mathbf{P(R_t)_m} = \mathbf{P(R)}_{1201 \times 1201} \cdot \mathbf{P(R_{t-1})_m} = \\ %
= \begin{Bmatrix}
P(0)_{0} & \cdots & P(0)_{12} \\ 
\vdots & \ddots & \vdots \\
P(12)_{0} & \cdots & P(12)_{12}
\end{Bmatrix}_{1201 \times 1201}
\cdot
\begin{Bmatrix}
P(R_{t-1})_0  \\
\vdots \\ 
P(R_{t-1})_{12}
\end{Bmatrix}_{1201 \times 1} = \\ %
= \begin{Bmatrix}
P(R_{t})_0 \\
\vdots \\ 
P(R_{t})_{12}
\end{Bmatrix}_{1201 \times 1}
\end{gathered}
\end{equation}

e si moltiplica per $\mathbf{P(k_t^*|R)}$ calcolato in precedenza, ottenendo così il numeratore della funzione $\eqref{eq:formula1}$:

\begin{equation}\label{eq:numeratoreok}
\mathbf{P(k_t^*|R)} \cdot \mathbf{P(R_t)_m}
\end{equation}

Il denominatore $P(k_t^*)$ è la sommatoria dei numeratori precedenti:

\begin{equation}\label{eq:denominatoreok}
P(k_t^*) = \sum_{i=0}^{t} \bigg( \mathbf{P(k_i^*|R)} \cdot \mathbf{P(R_i)_m} \bigg)
\end{equation}

ottenendo dunque per ogni giorno $t$ la probabilità a posteriori di $R_t$ dato $k_t$ (tabella $\ref{tab:prkfull}$ e figure $\ref{fig:prk}$ e $\ref{fig:prkmatrix}$):

\begin{equation}\label{eq:finale}
\mathbf{P(R_t|k_t^*)} = 
\left\{
\frac{ \mathbf{P(k_t^*|R)} \cdot \mathbf{P(R_t)_m} }{ 
\sum_{i=0}^{t} \bigg( \mathbf{P(k_i^*|R)} \cdot \mathbf{P(R_i)_m} \bigg)
}
\right\}_{1201 \times \omega}
\end{equation}

In [25]:
# We said we'd keep track of the sum of the log of the probability
# of the data for maximum likelihood calculation.
log_likelihood = 0.0

for previous_day, current_day in zip(k_bra.index[:-1], k_bra.index[1:]):

    #print(posteriors[previous_day])
    #(5a) Calculate the new prior
    current_prior = process_matrix @ posteriors[previous_day]

    #(5b) Calculate the numerator of Bayes' Rule: P(k|R_t)P(R_t)
    numerator = likelihoods[current_day] * current_prior

    #(5c) Calcluate the denominator of Bayes' Rule P(k)
    denominator = np.sum(numerator)

    # Execute full Bayes' Rule
    posteriors[current_day] = numerator/denominator
    
    # Add to the running sum of log likelihoods
    log_likelihood += np.log(denominator)

In [26]:
display(
    Latex(
        fr"""
\begin{{table}}
  \begin{{center}}
    \caption{{Estratto di matrice finale $\mathbf{{P(R_t|k_t^*)}}$: ${len(posteriors)} \times {len(posteriors.iloc[0])}$}}
    \label{{tab:prkfull}}
    \begin{{tabular}}{{c|c|c|c|c|c|c|c|}}
          & {posteriors.columns[0].strftime('%Y-%m-%d')} & 
          {posteriors.columns[1].strftime('%Y-%m-%d')} &
          {posteriors.columns[2].strftime('%Y-%m-%d')} &
          $\cdots$ & 
          {posteriors.columns[-3].strftime('%Y-%m-%d')} & 
          {posteriors.columns[-2].strftime('%Y-%m-%d')} & 
          {posteriors.columns[-1].strftime('%Y-%m-%d')} \\
        \toprule
        $R=0.00$ & {posteriors.iloc[0,0]:.2e} & {posteriors.iloc[0,1]:.2e} & {posteriors.iloc[0,2]:.2e} & $\cdots$ & {posteriors.iloc[0,-3]:.2e} & {posteriors.iloc[0,-2]:.2e} & {posteriors.iloc[0,-1]:.2e} \\
        \midrule
        $R=0.01$ & {posteriors.iloc[1,0]:.2e} & {posteriors.iloc[1,1]:.2e} & {posteriors.iloc[1,2]:.2e} & $\cdots$ & {posteriors.iloc[1,-3]:.2e} & {posteriors.iloc[1,-2]:.2e} & {posteriors.iloc[1,-1]:.2e} \\
        \midrule
        $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ \\
        \midrule
        $R=11.99$ & {posteriors.iloc[1199,0]:.2e} & {posteriors.iloc[1199,1]:.2e} & {posteriors.iloc[1199,2]:.2e} & $\cdots$ & {posteriors.iloc[1199,-3]:.2e} & {posteriors.iloc[1199,-2]:.2e} & {posteriors.iloc[1199,-1]:.2e} \\
        \midrule
        $R=12.00$ & {posteriors.iloc[1200,0]:.2e} & {posteriors.iloc[1200,1]:.2e} & {posteriors.iloc[1200,2]:.2e} & $\cdots$ & {posteriors.iloc[1200,-3]:.2e} & {posteriors.iloc[1200,-2]:.2e} & {posteriors.iloc[1200,-1]:.2e} \\
        \bottomrule
    \end{{tabular}}
  \end{{center}}
\end{{table}}
        """
    )
)

<IPython.core.display.Latex object>

In [27]:
fig, ax = plt.subplots(figsize=defaultFigsize)
for x in range(5):
    ax.plot(
        posteriors.iloc[:, x].index,
        posteriors.iloc[:, x].values,
        label=f"$t=${posteriors.iloc[:, x].name.strftime('%Y-%m-%d')}"
    )
ax.legend(loc="best")
ax.set_xlim(0,5)
ax.set_title("$P(R_t|k_t)$")
ax.set_xlabel("$R$")
latexplot(plt, "prk", r"Ditribuzione di probabilità $P(R_t|k_t)$ dei primi 5 giorni di dati in Italia. " + 
          "Si nota come nel primo giorno, non avendo alcuna informazione la probabilità è equamente distribuita "+
          "e successivamente ogni giorno diminuisca la deviazione standard e dunque si guadagni progressivamente in confidenza sul valore di $R_t$."
         )

<IPython.core.display.Latex object>

In [47]:
from matplotlib import cm
fig, ax = plt.subplots(figsize=(7,7))
im = ax.matshow(posteriors, aspect=posteriors.columns.size/400, cmap="Blues")
ax.set_ylim(400, 0)
ax.set_yticks([0, 100, 200, 300, 400])
ax.set_yticklabels([0, 1, 2, 3, 4])
ax.axhline(100, ls=":", c="k")
ax.set_ylabel("$R_t$", fontsize="xx-large")
ax.set_xlabel("giorni da $t_0$", fontsize="xx-large")
ax.set_title("Matrice $\mathbf{{P(R_t|k_t^*)}}$")
plt.colorbar(im)
latexplot(plt, "prkmatrix", "Matrice $\mathbf{{P(R_t|k_t^*)}}$. All'asse $y$ il valore di $R_t$ è stato limitato graficamente a 4. "+
          "Il colore indica la densità di probabilità.", showme=False
         )

<IPython.core.display.Latex object>

In [29]:
fig, ax = plt.subplots(figsize=(6, 3.5))
ax.set_xlim(0, 6)
ax.set_ylim(0, .05)
ax.set_xlabel("$R_t$")
line, = ax.plot([], [])
plt.close(fig)

def init():
    ax.plot([], [])

def animate(i):
    line.set_data(posteriors.iloc[:, i].index, posteriors.iloc[:, i].values)
    line.set_label(f"$t=${posteriors.iloc[:, i].name.strftime('%Y-%m-%d')}")
    ax.legend(loc="upper right", fontsize=12)
    return (line,)


ax.set_title("$P(R_t|k_t)$")

anim = FuncAnimation(fig, animate, init_func=init, interval=250, frames=posteriors.columns.size);

#HTML(anim.to_jshtml())

# Intervallo di credibilità

Data una distribuzione di probabilità $f(x)$ di posteriori Bayesiani della variabile aleatoria discreta $x$, l'intervallo di credibilità $\mathbf{HDI}_p$ (High Density Interval) corrispondente alla regione a maggior densità di probabilità, scelto il valore di $p\in(0,1)$ è

\begin{equation}\label{eq:hdi}
\mathbf{HDI}_p = [a, b] \in x, a < b \; : \; f(a) \simeq f(b), \; \int_{a}^{b} f(x) \simeq p
\end{equation}

Esistono diversi metodi per il calcolo di $\mathbf{HDI}_p$ ed i vantaggi/svantaggi di ciascuno dipendono dal tipo di studio effettuato <cite data-cite="hyndman1996computing"></cite>.

In generale, laddove siano noti solo i valori 
$\{x,f(x)\}$ ma non la funzione di distribuzione, è possibile calcolare l'intervallo integrando con regola di Simpson a partire dal valore massimo di 
$f(x)$ fino a che 

$$
\int_{a}^{b} \{x,f(x)\} < p
$$

ottenenendo così il minor intervallo 
$[a,b]$ tale per cui l'area sottesa dalla curva sia "il più possibile" vicina a 
$p$.

Un metodo equivalente 
<cite data-cite="k-sys"></cite> 
consiste nel calcolare la probabilità cumulativa

\begin{equation}\label{eq:cumf}
F = \sum f(x)
\end{equation}

ottenere la matrice quadrata antisimmetrica

\begin{equation}\label{eq:deltaf}
\Delta\mathbf{F} = F - F^{T} = 
\begin{Bmatrix}
F_{0} - F_{0} & F_{1} - F_{0} & \cdots & F_{|x|-1} - F_{0} & F_{|x|} - F_{0} \\
F_{0} - F_{1} & F_{1} - F_{1} & \cdots & F_{|x|-1} - F_{1} & F_{|x|} - F_{1} \\
\vdots & \vdots & \vdots & \vdots & \vdots \\
F_{0} - F_{|x|-1} & F_{1} - F_{|x|-1} & \cdots & F_{|x|-1} - F_{|x|-1} & F_{|x|} - F_{|x|-1} \\
F_{0} - F_{|x|} & F_{1} - F_{|x|} & \cdots & F_{|x|-1} - F_{|x|} & F_{|x|} - F_{|x|} \\
\end{Bmatrix}_{|x| \times |x|}
\end{equation}

e gli elementi $\Delta\mathbf{F}_{n,m}>p$ di riga $n$ e colonna $m$.

Gli indici dell'elemento tale per cui è minore la differenza tra $m$ (limite superiore) ed $n$ (limite inferiore)  rappresentano l'intervallo di credibilità, ovvero

\begin{equation}\label{eq:hdimatrix}
\mathbf{HDI}_p = [n, m] \in x, n < m \; : \; \min(m - n), \; \forall n, m \in \Delta\mathbf{F}_{n,m}>p
\end{equation}

Sia data ad esempio una distribuzione di probabilità con asimmetria positiva ($SK=3$, $\mu=7$, $\sigma=6$) della variabile aleatoria discreta $x \in [0,30]$ (figura $\ref{fig:hdiesempio}$).
Fissando $p=.9$ l'intervallo di credibilità Bayesiano è dato dalla curva sottesa tra $x=5$ e $x=18$.

Utilizzando il metodo della matrice antisimmetrica, si calcola la probabilità cumulativa $F$ e la matrice $\Delta\mathbf{F}$ (figura $\ref{fig:matrixdeltaf}$). Gli elementi $\Delta\mathbf{F}_{n,m}>.9$ sono delimitati dall'area blu in figura $\ref{fig:matrixdeltaf}$. L'elemento che soddisfa la $\eqref{eq:hdimatrix}$ è $\Delta\mathbf{F}_{5,18}=0.91$ (quadrato giallo in figura $\ref{fig:matrixdeltaf}$).

In [30]:
num = 30; skew = 3; mean = 7; std = 6; p = .9
x = np.linspace(0, num, num+1)

PAIRS = []
sol = None

sy = sps.skewnorm.pdf(x, skew, mean, std)
y = sps.skewnorm.cdf(x, skew, mean, std)

for i in np.linspace(sy[sy.argmax()], 0, 100):
    points = (sy >= i).nonzero()[0]
    PAIRS.append(
        [min(points), max(points)]
    )
    #ar2 = integrate.quad(lambda x: sps.skewnorm.pdf(x, skew, mean, std), min(points), max(points))
    ar = integrate.simps(sy[min(points):max(points)+1])
    if ar > p and not sol:
        sol = [i, min(points), max(points), ar]

fig, ax = plt.subplots(figsize=defaultFigsize)
plt.plot(sy, lw=1, marker="o")
verts = [
    (sol[1], 0),
    *zip(x[sol[1]:sol[2]+1], sps.skewnorm.pdf(x[sol[1]:sol[2]+1], skew, mean, std)), 
    (sol[2], 0)
]
poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
ax.add_patch(poly)
plt.axvline(mean+skew, ls=":", c="k")
plt.axvline(sol[1], c="r")
plt.axvline(sol[2], c="r")
plt.axhline(sol[0], ls="--", c="grey")
ax.set_ylim(0, max(sy)*1.1)
ax.set_title(f"$p={p}$    AREA$={sol[3]:.2f}$    $\mathbf{{HDI}}=({sol[1]},{sol[2]})$")
ax.set_xlabel("$x$", fontsize="x-large")
ax.set_ylabel("$f(x)$", fontsize="x-large")
plt.grid()
latexplot(
    plt, "hdiesempio",
    "Esempio di distrbuzione asimmetrica positiva e intervallo di credibilità $p>.9$."
)

<IPython.core.display.Latex object>

In [41]:
matrix = y - y[:, None]

fig, ax = plt.subplots(1, 2, figsize=(15,7))

im0 = ax[0].matshow(matrix, cmap="RdBu")
ax[0].plot(mean+skew, mean+skew, marker="s", c="g", markersize=6)
ax[0].axvline(mean+skew, ls="--", c="g")
ax[0].axhline(mean+skew, ls="--", c="g")
ax[0].set_xticks(x-.5)
ax[0].set_xticklabels([f"      {i}" for i in range(num+1)], fontsize="x-small")
ax[0].set_yticks(x-.5)
ax[0].set_yticklabels([f"\n\n{i}" for i in range(num+1)], fontsize="x-small")
ax[0].set_xlim(-0.5, num+.5)
ax[0].set_ylim(num+.5, -0.5)
ax[0].set_title("Esempio Matrice $\Delta\mathbf{F}$")
divider0 = make_axes_locatable(ax[0])
cax0 = divider0.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im0, cax=cax0)
ax[0].grid()

im1 = ax[1].matshow(pd.DataFrame(matrix > p), cmap="Blues")
ax[1].plot(mean+skew, mean+skew, marker="s", c="g", markersize=6)
ax[1].axvline(mean+skew, ls="--", c="g")
ax[1].axhline(mean+skew, ls="--", c="g")
ax[1].set_xticks(x-.5)
ax[1].set_xticklabels([f"      {i}" for i in range(num+1)], fontsize="x-small")
ax[1].set_yticks(x-.5)
ax[1].set_yticklabels([f"\n\n{i}" for i in range(num+1)], fontsize="x-small")
ax[1].plot(18, 5, marker="s", c="yellow", markersize=6)
ax[1].set_xlim(-0.5, num+.5)
ax[1].set_ylim(num+.5, -0.5)
ax[1].set_title("Esempio Matrice $\Delta\mathbf{F}>.90$")
divider1 = make_axes_locatable(ax[1])
cax1 = divider1.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im1, cax=cax1, extend="min")
ax[1].grid()
latexplot(
    plt, "matrixdeltaf",
    "Esempio di matrice $\Delta\mathbf{F}$ per $x \in [0,30]$. Le linee verdi indicano la mediana. " +
    "A sinistra la matrice completa, a destra la matrice per $\Delta\mathbf{F}>.90$. " +
    r"Gli estratti di matrice corrispondenti alla regione di interesse sono le tabelle $\ref{tab:deltaf}$ e $\ref{tab:deltafp}$.",
    showme=False
)

<IPython.core.display.Latex object>

In [32]:
display(
    Latex(
        fr"""
\begin{{table}}
  \begin{{center}}
    \caption{{Estratto di matrice $\Delta\mathbf{{F}}$}}
    \label{{tab:deltaf}}
    \resizebox{{\textwidth}}{{!}} {{
    \begin{{tabular}}{{{"c|" * 16}}}
        & {" & ".join([f"{i+16}" for i in range(15)])} \\
        \toprule
        0 & {" & ".join([f"{matrix[0,i+16]:.2f}" for i in range(15)])} \\
        \midrule
        1 & {" & ".join([f"{matrix[1,i+16]:.2f}" for i in range(15)])} \\
        \midrule
        2 & {" & ".join([f"{matrix[2,i+16]:.2f}" for i in range(15)])} \\
        \midrule
        3 & {" & ".join([f"{matrix[3,i+16]:.2f}" for i in range(15)])} \\
        \midrule
        4 & {" & ".join([f"{matrix[4,i+16]:.2f}" for i in range(15)])} \\
        \midrule
        5 & {" & ".join([f"{matrix[5,i+16]:.2f}" for i in range(15)])} \\
        \midrule
        6 & {" & ".join([f"{matrix[6,i+16]:.2f}" for i in range(15)])} \\
        \midrule
        7 & {" & ".join([f"{matrix[7,i+16]:.2f}" for i in range(15)])} \\
        \midrule
        \bottomrule
    \end{{tabular}} }}
  \end{{center}}
\end{{table}}
        """
    )
)

<IPython.core.display.Latex object>

In [33]:
matrix_p = matrix > p
display(
    Latex(
        fr"""
\begin{{table}}
  \begin{{center}}
    \caption{{Estratto di matrice $\Delta\mathbf{{F}}>p$}}
    \label{{tab:deltafp}}
    \resizebox{{\textwidth}}{{!}} {{
    \begin{{tabular}}{{{"c|" * 16}}}
        & {" & ".join([f"{i+16}" for i in range(15)])} \\
        \toprule
        0 & {" & ".join([f"{matrix_p[0,i+16]}" for i in range(15)])} \\
        \midrule
        1 & {" & ".join([f"{matrix_p[1,i+16]}" for i in range(15)])} \\
        \midrule
        2 & {" & ".join([f"{matrix_p[2,i+16]}" for i in range(15)])} \\
        \midrule
        3 & {" & ".join([f"{matrix_p[3,i+16]}" for i in range(15)])} \\
        \midrule
        4 & {" & ".join([f"{matrix_p[4,i+16]}" for i in range(15)])} \\
        \midrule
        5 & {" & ".join([f"{matrix_p[5,i+16]}" for i in range(15)])} \\
        \midrule
        6 & {" & ".join([f"{matrix_p[6,i+16]}" for i in range(15)])} \\
        \midrule
        7 & {" & ".join([f"{matrix_p[7,i+16]}" for i in range(15)])} \\
        \midrule
        \bottomrule
    \end{{tabular}} }}
  \end{{center}}
\end{{table}}
        """
    )
)

<IPython.core.display.Latex object>

In [34]:
def highest_density_interval(pmf, p=.9, debug=False, ax=None):
    # If we pass a DataFrame, just call this recursively on the columns
    if(isinstance(pmf, pd.DataFrame)):
        #fig, ax = plt.subplots()
        #ax.set_xlim(0,400)
        return pd.DataFrame([highest_density_interval(pmf[col], p=p, ax=ax) for col in pmf],
                            index=pmf.columns)
    
    cumsum = np.cumsum(pmf.values)
    
    # N x N matrix of total probability mass for each low, high
    total_p = cumsum - cumsum[:, None]
    
    # Return all indices with total_p > p
    lows, highs = (total_p > p).nonzero()
    
    # Find the smallest range (highest density)
    best = (highs - lows).argmin()
    
    low = pmf.index[lows[best]]
    high = pmf.index[highs[best]]

    if str(pmf.name) == "2020-02-22 18:00:00":
        fig, ax = plt.subplots(2, 2, figsize=(15, 15))
        # funzione densità
        ax[0,0].plot(cumsum, c="b", lw=5, label="probabilità cumulativa")
        ax1 = ax[0,0].twinx()
        ax1.plot(pmf.values*25, c="r", lw=3, label="densità di probabilità")
        ax1.set_ylim(0,.25)
        ax[0,0].set_xlim(0, 1200)
        ax[0,0].set_ylim(0, 1.1)
        ax[0,0].axvline(pmf.idxmax()*100, ls=":", c="k")
        ax[0,0].set_xticklabels([0, 2, 4, 6, 8, 10, 12])
        ax[0,0].legend(loc="upper right")
        ax1.legend(loc="lower right")
        ax[0,0].tick_params(axis="y", labelcolor="b")
        ax1.tick_params(axis="y", labelcolor="r")
        ax[0,0].grid(color="b", alpha=.25)
        ax1.grid(color="r", alpha=.25)
        ax[0,0].text(pmf.idxmax()*100-10, 1, f"$\mu={pmf.idxmax()}$", ha="right", va="bottom", fontsize="large")
        ax[0,0].set_title(f"Probabilità $R_t$ {pmf.name.date()}")
        # matrice
        im = ax[0,1].matshow(total_p, cmap='RdBu')
        #ax[0,1].xaxis.tick_bottom()
        ax[0,1].axhline(pmf.idxmax()*100, ls=":", c="k")
        ax[0,1].axvline(pmf.idxmax()*100, ls=":", c="k")
        ax[0,1].scatter(pmf.idxmax()*100, pmf.idxmax()*100, c="w")
        ax[0,1].text(600, pmf.idxmax()*100+10, f"$\mu={pmf.idxmax()}$", c="k", fontsize="large", va="top")
        ax[0,1].text(pmf.idxmax()*100+10, 600, f"$\mu={pmf.idxmax()}$", c="k", fontsize="large", ha="left")
        #ax[0,1].plot([0,0], [1201,1201], c="y")
        ax[0,1].set_xticklabels([None, 0, 2, 4, 6, 8, 10, 12])
        ax[0,1].set_yticklabels([None, 0, 2, 4, 6, 8, 10, 12])
        ax[0,1].set_xlabel("Limite Superiore", fontsize="large")
        ax[0,1].set_ylabel("Limite Inferiore", fontsize="large")
        ax[0,1].set_title(f"Matrice $\Delta\mathbf{{F_t}}$ {pmf.name.date()}")
        # p > .99
        ax[1,0].matshow(total_p > p, cmap='RdBu')
        #ax[1,0].xaxis.tick_bottom()
        ax[1,0].set_xticklabels([None, 0, 2, 4, 6, 8, 10, 12])
        ax[1,0].set_yticklabels([None, 0, 2, 4, 6, 8, 10, 12])
        ax[1,0].axhline(pmf.idxmax()*100, ls=":", c="k")
        ax[1,0].axvline(pmf.idxmax()*100, ls=":", c="k")
        ax[1,0].scatter(pmf.idxmax()*100, pmf.idxmax()*100, c="w")
        ax[1,0].axhline(lows[-1], c="r")
        ax[1,0].axvline(highs[0], c="c")
        ax[1,0].text(highs[0]+(1200-highs[0])/2, lows[-1]-10, f"{lows[-1]/100:.2f}", c="r", fontsize="large", va="bottom")
        ax[1,0].text(highs[0]+10, lows[-1]/2, f"{highs[0]/100:.2f}", c="c", fontsize="large", ha="left")
        ax[1,0].set_xlabel("Limite Superiore", fontsize="large")
        ax[1,0].set_ylabel("Limite Inferiore", fontsize="large")
        ax[1,0].set_title(f"$p>{p}$")
        # highest density
        ax[1,1].matshow(total_p > p, cmap='RdBu')
        #ax[1,1].xaxis.tick_bottom()
        ax[1,1].set_xticklabels([None, 0, 2, 4, 6, 8, 10, 12])
        ax[1,1].set_yticklabels([None, 0, 2, 4, 6, 8, 10, 12])
        ax[1,1].axhline(pmf.idxmax()*100, ls=":", c="k")
        ax[1,1].axvline(pmf.idxmax()*100, ls=":", c="k")
        ax[1,1].scatter(pmf.idxmax()*100, pmf.idxmax()*100, c="w")
        ax[1,1].axhline(lows[best], c="r")
        ax[1,1].axvline(highs[best], c="c")
        ax[1,1].scatter(highs[best], lows[best], color="w")
        ax[1,1].text(highs[best]+(1200-highs[best])/2, lows[best]-10, f"{lows[best]/100:.2f}", c="r", fontsize="large", ha="center", va="bottom")
        ax[1,1].text(highs[best]+10, lows[best]/2, f"{highs[best]/100:.2f}", c="c", fontsize="large", ha="left", va="center")
        ax[1,1].set_xlabel("Limite Superiore", fontsize="large")
        ax[1,1].set_ylabel("Limite Inferiore", fontsize="large")
        ax[1,1].set_title("Intervallo a maggior densità")
        divider = make_axes_locatable(ax[0,1])
        cax = divider.append_axes("right", size="5%", pad=0.05)
        im.set_clim(-1, 1)
        plt.colorbar(im, cax=cax, cmap='RdBu')
        plt.show()
        plt.close()
    
    return pd.Series([low, high],
                     index=[f'Low_{p*100:.0f}',
                            f'High_{p*100:.0f}'])

In [35]:
hdis = highest_density_interval(posteriors, p=.9)

In [36]:
most_likely = posteriors.idxmax().rename('ML')

# Look into why you shift -1
result = pd.concat([most_likely, hdis], axis=1)

# Grafico

Ottenuti i valori dell'intervallo di credibilità di $R_t$ per ogni $t$ (tabella $\ref{tab:rt}$ e figura $\ref{fig:rtbars}$) dalla distribuzione di probabilità della matrice $\mathbf{P(R_t|k_t^*)}$ si può quindi infine produrre un grafico dell'andamento temporale di $R_t$ (figura $\ref{fig:rtitalia}$).

In [37]:
display(
    Latex(
        fr"""
\begin{{table}}
  \begin{{center}}
    \caption{{Estratto di tabella dei valori $R_t$: ${len(result)} \times {len(result.iloc[0])}$}}
    \label{{tab:rt}}
    \begin{{tabular}}{{c|c|c|c|}}
        & Lo 90 & max & Hi 90 \\
        \toprule
        {result.index[0].strftime("%Y-%m-%d")} & {result.iloc[0,1]:.2f} & {result.iloc[0,0]:.2f} & {result.iloc[0,2]:.2f} \\
        \midrule
        {result.index[1].strftime("%Y-%m-%d")} & {result.iloc[1,1]:.2f} & {result.iloc[1,0]:.2f} & {result.iloc[1,2]:.2f} \\
        \midrule
        {result.index[2].strftime("%Y-%m-%d")} & {result.iloc[2,1]:.2f} & {result.iloc[2,0]:.2f} & {result.iloc[2,2]:.2f} \\
        \midrule
        $\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ \\
        \midrule
        {result.index[-3].strftime("%Y-%m-%d")} & {result.iloc[-3,1]:.2f} & {result.iloc[-3,0]:.2f} & {result.iloc[-3,2]:.2f} \\
        \midrule
        {result.index[-2].strftime("%Y-%m-%d")} & {result.iloc[-2,1]:.2f} & {result.iloc[-2,0]:.2f} & {result.iloc[-2,2]:.2f} \\
        \midrule
        {result.index[-1].strftime("%Y-%m-%d")} & {result.iloc[-1,1]:.2f} & {result.iloc[-1,0]:.2f} & {result.iloc[-1,2]:.2f} \\
        \bottomrule
    \end{{tabular}}
  \end{{center}}
\end{{table}}
        """
    )
)

<IPython.core.display.Latex object>

In [38]:
first = 1
last = 31
err = result[['Low_90', 'High_90']][first:last].sub(result['ML'][first:last], axis=0).abs()
fig, ax = plt.subplots(figsize=defaultFigsize)
ax.bar(
    [d.date() for d in result.index[first:last]],
    result["ML"][first:last],
    width=.75, alpha=.75, color="grey",
    capsize=2,
    error_kw={'alpha':.5, 'lw':1},
    yerr=err.values.T
)
ax.axhline(1.0, linestyle=':', color='k', lw=1)
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
ax.xaxis.set_minor_locator(mdates.DayLocator())
ax.xaxis.set_minor_formatter(mdates.DateFormatter('%d'))
ax.set_title(fr"Valore $R_t$ in Italia per i primi {last} giorni.")
latexplot(plt, "rtbars", fr"Valore $R_t$ in Italia per i primi {last} giorni con itervallo di confidenza al 90\%. " + 
          f"Il primo giorno {result.index[0].date()}, non avendo informazioni sui precedenti, serve solo da precedente per il calcolo della probabilità a posteriori del giorno successivo."
         )

<IPython.core.display.Latex object>

In [39]:
#result

In [40]:
def plot_rt(result, ax, state_name):
    
    ax.set_title(f"{state_name}")
    
    # Colors
    ABOVE = [1,0,0]
    MIDDLE = [1,1,1]
    BELOW = [0,0,0]
    cmap = ListedColormap(np.r_[
        np.linspace(BELOW,MIDDLE,25),
        np.linspace(MIDDLE,ABOVE,25)
    ])
    color_mapped = lambda y: np.clip(y, .5, 1.5)-.5
    
    index = result['ML'].index.get_level_values('data')
    values = result['ML'].values
    
    # Plot dots and line
    ax.plot(index, values, c='k', zorder=1, alpha=.25)
    ax.scatter(index,
               values,
               s=40,
               lw=.5,
               c=cmap(color_mapped(values)),
               edgecolors='k', zorder=2)
    
    # Aesthetically, extrapolate credible interval by 1 day either side
    lowfn = interp1d(date2num(index),
                     result['Low_90'].values,
                     bounds_error=False,
                     fill_value='extrapolate')
    
    highfn = interp1d(date2num(index),
                      result['High_90'].values,
                      bounds_error=False,
                      fill_value='extrapolate')
    
    extended = pd.date_range(start=index[1],
                             end=index[-1]+pd.Timedelta(days=1))
    
    ax.fill_between(extended,
                    lowfn(date2num(extended)),
                    highfn(date2num(extended)),
                    color='k',
                    alpha=.1,
                    lw=0,
                    zorder=3)

    ax.axhline(1.0, c='k', lw=1, label='$R_t=1.0$', alpha=.25);
    
    # Formatting
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.set_minor_locator(mdates.DayLocator())
    
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.1f}"))
    ax.yaxis.tick_right()
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.margins(0)
    ax.grid(which='major', axis='y', c='k', alpha=.1, zorder=-2)
    ax.margins(0)
    ax.set_ylim(0.0, 4.0)
    ax.set_xlim(index[1], result.index.get_level_values('data')[-1]+pd.Timedelta(days=1))
    ax.axvline(pd.Timestamp("2020-03-11"), ls=":", c="k")
    ax.axvline(pd.Timestamp("2020-05-04"), ls=":", c="k")
    ax.fill_between(
        [pd.Timestamp("2020-03-11"), pd.Timestamp("2020-05-04")],
        0, 4,
        color="k", alpha=.01
    )
    ax.text(
        pd.Timestamp("2020-04-07"), 3,
        "Lockdown nazionale", ha="center"
    )
    fig.set_facecolor('w')

state_name = "Italia"
fig, ax = plt.subplots(figsize=defaultFigsize)

plot_rt(result, ax, state_name)
ax.set_title(f'$R_t$ per {state_name}')
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
latexplot(plt, "rtitalia", r"Valori di $R_t$ in Italia con intervallo di credibilità al 90\%. "+
          "L'area ombreggiata delimitata dalle linee verticali indica il periodo di lockdown nazionale completo.",
          showme=False
         )

<IPython.core.display.Latex object>

# Conclusioni

È dunque possibile stimare il valore del numero di riproduzione effettivo $R_t$ in base al numero noto di nuovi casi giornalieri $k_t$ e al tempo seriale $T_{serial}$ con metodo Bayesiano $P(R_t|k_t)$, utilizzando i risultati del giorno precedente come base per il calcolo della probabilità a posteriori, scegliendo l'intervallo di credibilità desiderato. È consigliabile smussare la curva dei nuovi casi (con media mobile) per diminuire la variabilità dovuta ad errori e correzioni nella collezione giornaliera dei dati.

# Codice Python

Si fornisce di seguito un esempio di codice `Python` per l'esecuzione del metodo illustrato per il calcolo di $R_t$ in Italia, adattato dalla versione di Kevin Systrom <cite data-cite="k-sys"></cite>.

```python
import numpy as np
import pandas as pd
import scipy.stats as sps

# DEF: Calcolo nuovi casi e media mobile
def smooth(y, std=2):
    dy = y.diff()
    dy_smoothed = dy.rolling(7,
            win_type="gaussian",
            min_periods=1,
            center=True).mean(std=std).round()
    return dy_smoothed, dy

# DEF: Estrazione intervallo di credibilità
def highest_density_interval(pmf, p=.9):
    if(isinstance(pmf, pd.DataFrame)):
        return pd.DataFrame(
            [highest_density_interval(
                pmf[col], p=p
            ) for col in pmf],
            index=pmf.columns)
    
    cumsum = np.cumsum(pmf.values)
    total_p = cumsum - cumsum[:, None]
    lows, highs = (total_p > p).nonzero()
    best = (highs - lows).argmin()
    low = pmf.index[lows[best]]
    high = pmf.index[highs[best]]
    
    return pd.Series(
        [low, high],
        index=[
            f"Low_{p*100:.0f}",
            f"High_{p*100:.0f}"])

########### CALCOLO ###########

# Intervallo seriale e gamma
T_serial = 7.5
gamma = 1 / T_serial

# Vettori R
Rt_min = 0
Rt_max = 12
Rt_bra = np.linspace(Rt_min, Rt_max, Rt_max*100+1)
Rt_ket = Rt_bra[:, None]

# Caricamento dati
url_base = "https://raw.githubusercontent.com/"
url_repo = "pcm-dpc/COVID-19/master/dati-andamento-nazionale/"
url_file = "dpc-covid19-ita-andamento-nazionale.csv" 
url_ita = url_base + url_repo + url_file
ita = pd.read_csv(
    url_ita,
    usecols=["data", "totale_casi"],
    parse_dates=["data"],
    index_col=["data"],
    squeeze=True).sort_index()

# Calcolo nuovi casi e media mobile
k_bra, orig = smooth(ita, std=2.5)

# Matrice lambda
lam = k_bra[:-1].values * np.exp(gamma * (Rt_ket - 1))

# Matrice verosimiglianze P(k*|R)
likelihoods = pd.DataFrame(
    data = sps.poisson.pmf(k_bra[1:].values, lam),
    index = Rt_bra,
    columns = k_bra.index[1:])

# Matrice P(R)
sigma = .25
process_matrix = sps.norm(
    loc=Rt_bra,
    scale=sigma
).pdf(Rt_ket)
process_matrix /= process_matrix.sum(axis=0)

# Primo precedente della matrice P(R|k)
prior0 = np.ones_like(Rt_bra)/len(Rt_bra)
prior0 /= prior0.sum()

# Matrice vuota P(R|k)
posteriors = pd.DataFrame(
    index=Rt_bra,
    columns=k_bra.index,
    data={k_bra.index[0]: prior0}
)

# Calcolo P(R|k*) giorno per giorno
for previous_day, current_day in zip(k_bra.index[:-1], k_bra.index[1:]):
    # Calcolo del precedente P(R_t) = P(R) * P(R_(t-1))
    current_prior = process_matrix @ posteriors[previous_day]
    # Calcolo numeratore P(k*|R)P(R_t) [equazione (2)]
    numerator = likelihoods[current_day] * current_prior
    # Calcolo denominatore P(k*) [equazione (2) e (3)]
    denominator = np.sum(numerator)
    # Calcolo della probabilità a posteriori P(k*|R_t) [equazione (2)]
    posteriors[current_day] = numerator/denominator

# Estrazione valori per intervallo di credibilità
hdis = highest_density_interval(posteriors, p=.9)

# Estrazione massima probabilità
most_likely = posteriors.idxmax().rename("ML")

# Tabella valori di R in t
result = pd.concat([most_likely, hdis], axis=1)
```